# Project Milestone - Nhóm 2

### Big Data Applications: Machine Learning at Scale

### Đề tài: Music recommendation system

### GV: Quách Đình Hoàng


### Members:

1. Nguyễn Anh Đắc - 19133020

2. Nguyễn Thanh Tân Kỷ - 19133031

3. Đào Thị Cẩm Tiên - 19133055

4. Lại Hữu Trác - 19133059

### 1. Thông tin về project

#### a. Bài toán mà nhóm muốn giải quyết

- Tạo một hệ thống đề xuất các bài hát cho người dùng dựa vào số lượng lượt nghe của các bài hát
- Các nhạc sĩ đang thịnh hành
- Hiển thị top các bài hát đang thịnh hành trên hệ thống

#### b. Tập dữ liệu, input/output của bài toán

- Tập dữ liệu: Million Song - Recommendation Engines (Gồm có "10000.txt" và "song_data.csv")
- Link dataset: https://www.kaggle.com/code/mgmarques/million-song-recommendation-engines/data
- Tập dữ liệu về user (10000.txt) có 3 cột và 2000000 dòng chứa số lượt phát của người dùng ẩn danh cho các bài hát có trong tập dữ liệu triệu bài hát
    + user_id: ID của người dùng
    + song_id: ID của bài hát
    + listen_count: Lượt nghe của bài hát
- Tập dữ liệu về bài hát (song_data.csv) có 5 cột và 1000000 dòng chứa các thuộc tính của một triệu bài hát
    + song_id: ID của bài hát
    + title: Tên bài hát
    + release: Tên ablum của bài hát đó
    + artist_name: Tên nghệ sĩ, ca sĩ
    + year: Năm bài hát được phát hành
- Input: Tập dữ liệu về user (10000.txt) và tập dữ liệu về bài hát (song_data.csv)
- Ouput: List bài hát được đề xuất cho user

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

data_song = pd.read_csv("song_data.csv")
data_song.columns

Index(['song_id', 'title', 'release', 'artist_name', 'year'], dtype='object')

In [2]:
# 3 dòng đầu tiên:
data_song.head(3)

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006


In [3]:
# 3 dòng cuối:
data_song.tail(3)

,song_id,title,release,artist_name,year
999997,SOHODZI12A8C137BB3,Novemba,Dub_Connected: electronic music,Gabriel Le Mar,0
999998,SOLXGOR12A81C21EB7,Faraday,The Trance Collection Vol. 2,Elude,0
999999,SOWXJXQ12AB0189F43,Fernweh feat. Sektion Kuchikäschtli,So Oder So,Texta,2004


In [4]:
data_song.shape

(1000000, 5)

In [5]:
data_user = pd.read_csv('10000.txt', sep='\t', header=None)
data_user.columns = ['user_id', 'song_id', 'listen_count']
data_user.head()
data_user.to_csv("out.csv")
data_user.head(10)

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODXRTY12AB0180F3B,1
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFGUAY12AB017B0A8,1
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFRQTD12A81C233C0,1
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOHQWYZ12A6D4FA701,1


In [6]:
data_user.shape

(2000000, 3)

#### c. Cách đánh giá kết quả

- Nhóm em sẽ thực hiện theo cách tiếp cận Collaborative filtering - xây dựng hệ thống gợi ý dựa trên các ngươi dùng tương đồng.
- Thuật toán sử dụng:
Alternating least squares

### 2. Giải pháp

In [7]:
import findspark
findspark.init()

import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max
from pyspark.ml.feature import  StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql import Row
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

In [8]:
spark = SparkSession.builder.appName("lastfm").getOrCreate()

In [9]:
file_path = './out.csv'
df_listenings_agg  = spark.read.format('csv').option('header',True).option('inferSchema',True).load(file_path)                    #spark.createDataFrame(data_user) #spark.read.text(file_path).option('header',True).option('inferSchema',True) #data frame, header will infer column types from csv
df_listenings_agg .show()
row = df_listenings_agg.count()
col = len(df_listenings_agg.columns)
print(row,col)

+---+--------------------+------------------+------------+
|_c0|             user_id|           song_id|listen_count|
+---+--------------------+------------------+------------+
|  0|b80344d063b5ccb32...|SOAKIMP12A8C130995|           1|
|  1|b80344d063b5ccb32...|SOBBMDR12A8C13253B|           2|
|  2|b80344d063b5ccb32...|SOBXHDL12A81C204C0|           1|
|  3|b80344d063b5ccb32...|SOBYHAJ12A6701BF1D|           1|
|  4|b80344d063b5ccb32...|SODACBL12A8C13C273|           1|
|  5|b80344d063b5ccb32...|SODDNQT12A6D4F5F7E|           5|
|  6|b80344d063b5ccb32...|SODXRTY12AB0180F3B|           1|
|  7|b80344d063b5ccb32...|SOFGUAY12AB017B0A8|           1|
|  8|b80344d063b5ccb32...|SOFRQTD12A81C233C0|           1|
|  9|b80344d063b5ccb32...|SOHQWYZ12A6D4FA701|           1|
| 10|b80344d063b5ccb32...|SOIYTOA12A6D4F9A23|           1|
| 11|b80344d063b5ccb32...|SOIZAZL12A6701C53B|           5|
| 12|b80344d063b5ccb32...|SOJNNUA12A8AE48C7A|           1|
| 13|b80344d063b5ccb32...|SOJPFQG12A58A7833A|           

In [10]:
row = df_listenings_agg.count()
col = len(df_listenings_agg.columns)
print(row,col)

2000000 4


In [11]:
df_listenings_agg = df_listenings_agg.limit(20000)

In [12]:
old_strindexer = [StringIndexer(inputCol = col, outputCol = col + '_index').fit(df_listenings_agg) for col in list(set(df_listenings_agg.columns)- set(['count']))]
indexer = [curr_strindexer.setHandleInvalid("keep") for curr_strindexer in old_strindexer]
pipeline = Pipeline(stages = indexer)
data_p = pipeline.fit(df_listenings_agg).transform(df_listenings_agg)
data_p.show(25)

+---+--------------------+------------------+------------+---------+-------------+-------------+------------------+
|_c0|             user_id|           song_id|listen_count|_c0_index|song_id_index|user_id_index|listen_count_index|
+---+--------------------+------------------+------------+---------+-------------+-------------+------------------+
|  0|b80344d063b5ccb32...|SOAKIMP12A8C130995|           1|      0.0|       2502.0|        111.0|               0.0|
|  1|b80344d063b5ccb32...|SOBBMDR12A8C13253B|           2|      1.0|       2550.0|        111.0|               1.0|
|  2|b80344d063b5ccb32...|SOBXHDL12A81C204C0|           1|  11112.0|        253.0|        111.0|               0.0|
|  3|b80344d063b5ccb32...|SOBYHAJ12A6701BF1D|           1|  12223.0|        212.0|        111.0|               0.0|
|  4|b80344d063b5ccb32...|SODACBL12A8C13C273|           1|  13334.0|        214.0|        111.0|               0.0|
|  5|b80344d063b5ccb32...|SODDNQT12A6D4F5F7E|           5|  14445.0|    

In [13]:
data_p = data_p.select('user_id_index', 'song_id_index', 'listen_count').orderBy('user_id_index')
data_p.show()

+-------------+-------------+------------+
|user_id_index|song_id_index|listen_count|
+-------------+-------------+------------+
|          0.0|         44.0|           1|
|          0.0|       2572.0|           2|
|          0.0|        578.0|           3|
|          0.0|        415.0|           1|
|          0.0|       1431.0|           1|
|          0.0|        580.0|           2|
|          0.0|        176.0|          20|
|          0.0|       4353.0|           1|
|          0.0|       4354.0|           1|
|          0.0|        419.0|           7|
|          0.0|       1444.0|           2|
|          0.0|       2523.0|           1|
|          0.0|       2529.0|           1|
|          0.0|        905.0|           2|
|          0.0|       2531.0|           1|
|          0.0|        251.0|          11|
|          0.0|       2539.0|           2|
|          0.0|       1456.0|           3|
|          0.0|       2557.0|           2|
|          0.0|       4417.0|           1|
+----------

In [14]:
(training, test) = data_p.randomSplit([0.5,0.5])

In [15]:
#USERID = "user_id_index"
#TRACK = "song_id_index"
#COUNT = "listen_count"

als = ALS(maxIter = 5, regParam = 0.01, userCol = "user_id_index", itemCol = "song_id_index", ratingCol = "listen_count")
# Alternating Least Squares algorithm
model = als.fit(training)
predictions = model.transform(test)

evaluator = RegressionEvaluator(metricName="rmse", labelCol="listen_count", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(rmse)

nan


In [16]:
recs = model.recommendForAllUsers(10)
recs.show()

D:\NAM4\pySpark\spark-3.2.2-bin-hadoop3.2\python\pyspark\sql\context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------------+--------------------+
|user_id_index|     recommendations|
+-------------+--------------------+
|            1|[{5351, 27.311214...|
|            3|[{6930, 23.619585...|
|            5|[{5635, 21.949518...|
|            6|[{3948, 34.17879}...|
|           12|[{4364, 26.419838...|
|           13|[{5010, 26.879093...|
|           15|[{1298, 27.217657...|
|           16|[{5028, 45.224716...|
|           19|[{3069, 30.724297...|
|           20|[{5635, 46.92202}...|
|           22|[{3069, 30.370766...|
|           26|[{3069, 61.34582}...|
|           27|[{3261, 28.767578...|
|           28|[{3988, 30.326637...|
|           31|[{1298, 34.157825...|
|           34|[{223, 24.380547}...|
|           40|[{1869, 55.610744...|
|           41|[{3988, 34.498547...|
|           43|[{3572, 23.043234...|
|           44|[{5635, 7.741981}...|
+-------------+--------------------+
only showing top 20 rows



In [17]:
recs.take(1)

[Row(user_id_index=1, recommendations=[Row(song_id_index=5351, rating=27.311214447021484), Row(song_id_index=411, rating=26.167922973632812), Row(song_id_index=5635, rating=26.006927490234375), Row(song_id_index=24, rating=23.47688865661621), Row(song_id_index=5, rating=21.97063446044922), Row(song_id_index=1005, rating=20.81513023376465), Row(song_id_index=1963, rating=19.567541122436523), Row(song_id_index=2984, rating=18.51605796813965), Row(song_id_index=3261, rating=17.493221282958984), Row(song_id_index=2860, rating=17.24549102783203)])]